In [15]:
import re
import emoji
import random
import pandas as pd
from pandas import read_csv
from opencc import OpenCC
# !pip install emoji

# Organizing Facebook Content

In [16]:
### load data from Fb_crawling_output
df = read_csv('3_fb_crawling_output.csv', encoding='utf8',index_col=0)
#df.head()

In [17]:
### Remove non-"店家" post ###
temp_p_lst = []
temp_d_lst = []
temp_p = ''
temp_d = ''

for post_content, date_time in zip(df['Description'].astype(str), df['published'].astype(str)):
    cc = OpenCC('s2tw') # 簡轉繁
    post_content = cc.convert(post_content)
    post = post_content.replace(' ','').replace('￼￼','').replace('￼￼￼','').replace('\n','') \
        .replace('$','').replace('／','').replace('…','').replace('：',':').replace(':',':').replace('%','') \
        .replace(u'\u3000',u'').replace('▎','').replace('）',')').replace('（','(').replace('O','') \
        .replace('【',' ').replace('】',':').replace('■','').replace('◆','').replace('●','').replace('★','') \
        .replace('1.','').replace('2.','').replace('3.','').replace('4.','').replace('5.','') \
        .replace('6.','').replace('7.','') \
        .replace('\'','').replace('_','').replace('☆','') \
        .replace('▎','').replace('🁢',' ').replace('-','').replace('◎','')\
        .replace('更多','').replace('分隔線','').replace('NT','') \
        .replace('▎店家:','店家:').replace('▎店　　名:','店家:') \
        .replace('店家/鄰近','店家:').replace('店家/鄰近地標:','店家:').replace('店家鄰近地點:','店家:') \
        .replace('店家/鄰近地點:','店家:') \
        .replace('店名:','店家:').replace('▎店　　家:','店家:').replace(' 店家','店家:')\
        .replace('【店家】','店家:').replace('店家名稱:','店家:').replace('店家資訊:','店家:') \
        .replace('鄰近:','%鄰近地點:').replace('所在地區','%鄰近地點:') \
        .replace('鄰近位置:','%鄰近地點:').replace('地點:','%鄰近地點:') \
        .replace('所在地區/鄰近位置:','%鄰近地點:') \
        .replace('鄰近地點','%鄰近地點').replace('臨近地點:','%鄰近地點:').replace('地址:','%鄰近地點:') \
        .replace('鄰近地區','%鄰近地點').replace('區域','%鄰近地點:').replace('地區/鄰近','%鄰近地點:')\
        .replace('位置:','%鄰近地點:').replace('位置','%鄰近地點:')\
        .replace('拉麵名稱','%.G拉麵名稱').replace('餐點名稱:','%.G拉麵名稱:')\
        .replace('餐點:','%.G拉麵名稱:').replace('拉麵品項:','%.G拉麵名稱')\
        .replace('品項:','%.G拉麵名稱:').replace('品名:','%.G拉麵名稱:')\
        .replace('名稱:','%.G拉麵名稱:').replace('品項價格:','%.G拉麵名稱:')\
        .replace('配置:','Z配置').replace('配　　置','Z配置').replace('配置(','Z配置').replace('▎配置:','Z配置')\
        .replace('心得感想:','Z').replace('感想:','Z').replace('心得:','Z') \
        .replace('完食感受:','Z').replace('完食感想:','Z') \
        .replace('味增','味噌').replace('食八番','食8番').replace('五ノ','五之').replace('総','總')
    
    if "店家" in post[:5]:
        temp_p_lst.append(post)

        for word in date_time:
            temp_d += word
            if word == '日':
                break
        if ('小時' in temp_d) or ('分鐘' in temp_d):
            temp_d = '2020年12月2日'   # Change the current date as nedded
        elif ('昨天' in temp_d):
            temp_d = '2020年12月1日'   # Change the date as nedded
        elif '年' not in temp_d:
            temp_d = '2020年' + temp_d      
        temp_d_lst.append(temp_d)
        temp_d = ''

In [18]:
### 有無標準發文格式分類 
unorganized_shops = []
unorganized_date = []
ramen_shop_raw = []
ramen_name_raw = []
ramen_review_raw = []
ramen_date_raw = []

for shops, date in zip(temp_p_lst, temp_d_lst):
    if ('%'not in shops or 'G' not in shops or 'Z' not in shops \
        or shops.index('Z')>shops.index('G')+80) :
        unorganized_shops.append(shops)
        unorganized_date.append(date)      
    else:
        ramen_shop_raw.append(shops[:shops.index('%')]) # 鄰近地點
        ramen_name_raw.append(shops[shops.index('G')+1:shops.index('Z')])
        ramen_review_raw.append(shops[shops.index('Z')+1:shops.index('Z')+265]+'...')
        ramen_date_raw.append(date)

In [19]:
#### second filtering
unorganized_unorganized_shops = []
unorganized_unorganized_date = []

for shops, date in zip(unorganized_shops, unorganized_date):
    if ('G' in shops and '0' in shops):
        ramen_shop_raw.append(shops[:shops.index('%')])
        ramen_name_raw.append(shops[shops.index('G')+1 : shops.index('G')+35])
        ramen_review_raw.append(shops[shops.index('G')+15 : shops.index('G')+285]+'...')
        ramen_date_raw.append(date)
    else:
        unorganized_unorganized_shops.append(shops)
        unorganized_unorganized_date.append(date)

In [20]:
### Dealing with shop_name
ramen_shop_list = []

for shops in ramen_shop_raw:
    shops.replace('店家','').replace('#','').replace('鄰近','')
    if ':' in shops:
        shops = shops[shops.index(':')+1:]
    shops = emoji.demojize(shops)
    shops = shops.replace('／','')
    shops = re.sub(':\S+?:', ' ', shops)
    if ':' in shops:
        shops = shops[:shops.index(':')]
    if len(shops) <= 1:
        shops = ''
    if '地址' in shops and '用餐' in shops:
        if shops.index('地') < shops.index('用'):
            ramen_shop_list.append(shops[:shops.index('地')])
        else:
            ramen_shop_list.append(shops[:shops.index('用')])
    elif '地址' in shops:
        ramen_shop_list.append(shops[:shops.index('地')])
    elif '用餐' in shops:
        ramen_shop_list.append(shops[:shops.index('用')])
    else:
        ramen_shop_list.append(shops)

In [21]:
### double check shop_name
ramen_shop_list_final = []

for shop in ramen_shop_list:
    shop = re.sub(r'[^\w\s]','',shop)
    shop = shop.replace('#','').replace('鄰近','').replace('臨近','').replace('排隊狀況','') \
    .replace('日期','').replace('店家','').replace('附近','').replace('《','').replace('》','') \
    .replace('。','').replace('、',' ').replace('｜','').replace('？','').replace('地區','') \
    .replace('(','').replace(')','').replace('·','').replace('/','').replace('地標','') \
    .replace('※','').replace('ぁ','あ').replace('²','2').replace(' ','')
    if len(shop) >= 31:
        shop = shop[:31]
    if '捷運' in shop:
        shop = shop[:shop.index('捷運')]
    ramen_shop_list_final.append(shop)  

In [22]:
### Dealing with ramen_name 
ramen_name_list = []
ramen_name_list_final = []

for names in ramen_name_raw:
    names = emoji.demojize(names)
    names = re.sub(':\S+?:', ' ', names)
    new_name = names.replace('拉麵%.G','').replace('%.G','').replace('#','').replace('鄰近','')
    last_ch = new_name[-1]
    first_ch = new_name[0]
    # ramen_name_list = []
    if ('0' in new_name) and ('00' not in new_name) and ('2020' not in new_name) \
        and (last_ch != ')'):
        if new_name[-1] == '元' or new_name[-2:] == '日元':
            ramen_name_list.append(new_name[:new_name.index('元')+1])
        elif new_name[-2:] == '日圓':
            ramen_name_list.append(new_name[:new_name.index('圓')+1])
        else:
            ramen_name_list.append(new_name[:new_name.index('0')+1])
    elif '00' in new_name and last_ch != ')' and '2020' not in new_name:
        ramen_name_list.append(new_name[:new_name.index('00')+2])
    elif first_ch != '拉' and '拉' in new_name:
        ramen_name_list.append(new_name[new_name.index('拉'):])
    elif '0' not in new_name and '00' not in new_name and '/' not in new_name\
        and last_ch.isdigit() == False and last_ch != '麵' \
        and last_ch != ')'and '+' not in new_name:
        new_point =  new_name.replace('麵','麵H').replace('拉麵H名稱','拉麵名稱')
        if 'H' in new_point:
            ramen_name_list.append(new_point[:new_point.index('H')])
        else:
            ramen_name_list.append(new_point)
    else:
        ramen_name_list.append(new_name)

In [23]:
### double check ramen_name
for name in ramen_name_list:
    if ':' in name:
        name = name[name.index(':')+1:]
    name = name.replace('拉麵名稱/價格','').replace('拉麵名稱','').replace('拉麵名稱價格','').replace('價格','')
    ramen_name_list_final.append(name)  

In [24]:
### Dealing with review
ramen_review_list = []
ramen_review_list_final = []

for name, reviews in zip(ramen_name_list, ramen_review_raw):
    new_reviews = reviews.replace('拉麵%.G','').replace('%.G','').replace('%','').replace('$','')
    first_few_words = new_reviews[0:6]
    if '配置'not in first_few_words:
        for i in range(-(len(name)),0,1):
            if (name[i:]) == (new_reviews[:-i]):
                updated_review = new_reviews[-i:]
                ramen_review_list.append(updated_review)
                break
            elif (i == -1) and (ramen_name_list[i:]) != (ramen_review_raw[:-i]):
                pattern="[\u4e00-\u9fa5]+" 
                regex = re.compile(pattern)
                results =  regex.findall(new_reviews)
                results_to_str =' '.join([str(elem) for elem in results]) 
                ramen_review_list.append(results_to_str)
    else:
        ramen_review_list.append(new_reviews)

In [25]:
### double check review
punc = [',' , '.' , '，', '。', ')',' ','……','；','、','\'','/','?','元']
for reviews in ramen_review_list:
    if reviews[0] in punc:
        reviews = reviews[1:]
    ramen_review_list_final.append(reviews)

In [27]:
### Create post_id 
post_id = ['t3'+ str(i) for i in range(len(ramen_shop_list))]
random.shuffle(post_id)
'''post_id = [i for i in range(len(ramen_shop_list))]
random.shuffle(post_id)'''

'post_id = [i for i in range(len(ramen_shop_list))]\nrandom.shuffle(post_id)'

In [29]:
### Create stem_store_name
stem_store = []

for store in ramen_shop_list_final:
    store = store.lower()
    store = re.sub(r'[^\w\s]','',store)
    store = store.replace('総','總').replace('麺','麵').replace('醤','醬').replace('らーめん七面','七面') \
                .replace('臺南白露','白露').replace('鷹流東京豚骨拉麵極匠','鷹流極匠') \
                .replace('太陽蕃茄EXPRESS','太陽蕃茄拉麵EXPRESS') \
                .replace('鷹流東京醬油拉麵蘭丸','鷹流蘭丸').replace('鷹流臺灣本店','鷹流拉麵臺灣本店') \
                .replace('ラーメン','拉麵').replace('らーめん','拉麵').replace('柑橘shin','柑橘shinn')  \
                .replace('麵屋壹之穴','麵屋壹之穴ichi').replace('の','之').replace('aqua2','') \
                .replace('麵鋪','麵舖').replace('台湾','臺灣').replace('番茄','蕃茄')  \
                .replace('台','臺').replace('麵や青鈴','麵屋青鈴')
    stem_store.append(store)

In [30]:
#### Output_csv
df_output = pd.DataFrame(list(zip(*[post_id, ramen_shop_list_final, stem_store, ramen_date_raw, ramen_name_list_final, ramen_review_list_final])))
col_names = ['post_id', 'stores', 'stem_store', 'create_on', 'ramen_name', 'fb_review']
df_output.columns = col_names
df_output["stores_len"] = df_output["stores"].str.len()
df_output["ramen_name_len"] = df_output["ramen_name"].str.len() 
df_output["fb_review_len"] = df_output["fb_review"].str.len() 
df_output_ = df_output[(df_output['stores_len'] > 1.0)]
df_output_ = df_output_[(df_output['ramen_name_len'] > 1.0)]
df_output_ = df_output_[(df_output['fb_review_len'] > 2.0)]

df_new = df_output_.sort_values(by=['stores', 'ramen_name_len'])
df_new = df_new.reset_index(drop=True)
df_new = df_new.drop(columns=['stores_len','ramen_name_len','fb_review_len'])
df_new = df_new.sort_index(axis=0 ,ascending=True)
df_new = df_new.iloc[::-1]
df_new = df_new.reset_index(drop=True)
df_new.to_csv('date_2_output_fb_crawling.csv', index=True)

/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


# Mapping store_name with Map (create store_id) -> TABLE3

In [553]:
import os, itertools
from itertools import groupby

In [809]:
df_new = read_csv('2_output_fb_crawling.csv', encoding='utf8',index_col=0)
stores_input = list(df_new['stem_store'])
shop_sorted = sorted(list(set(stores_input)))

In [783]:
util_func = lambda x: x[0]  
first_sort = [list(ele) for i, ele in groupby(shop_sorted, util_func)] 
lst = list(itertools.chain(*first_sort))
#first_sort

In [784]:
### SECOND GROUPING
unique = []
second_sort = []

util_func = lambda x: x[1]
for item in first_sort:
    if len(item) > 1:
        temp = sorted(item, key = util_func)
        second_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
    else:
        unique.append(item[0])

second_sort_len = []

for item_1 in second_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            second_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [786]:
### THIRD GROUPING
third_sort = []

util_func = lambda x: x[2]
for item in second_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' or item[0][:4] == '太陽' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        third_sort.append([item])   
    else:
        if len(item) > 1 and len(item[0]) >= 3:
            temp = sorted(item, key = util_func)
            third_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
        elif len(item) > 1 and len(item[0]) == 2:
            third_sort.append([item])
        else:
            unique.append(item[0])
        
third_sort_len = []

for item_1 in third_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            third_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [788]:
### FOURTH GROUPING
fourth_sort = []

util_func = lambda x: x[3]
for item in third_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' or item[0][:4] == '太陽' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        fourth_sort.append([item])   
    else:
        if len(item) > 1 and len(item[0]) >= 4:
            temp = sorted(item, key = util_func)
            fourth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 3:
            fourth_sort.append([item])
        else:
            unique.append(item[0])

fourth_sort_len = []

for item_1 in fourth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fourth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [790]:
done = []
unclassified_group = []

for item in fourth_sort_len:
    grouped = []
    unclassified = []
    grouped.append(item[0])
    for i in range(len(item)-1):
        if (item[0] in item[i+1]) or (item[0][:6] in item[i+1]):
            grouped.append(item[i+1])
        else:
            unclassified.append(item[i+1])
    done.append(grouped)
    if unclassified != []:
        unclassified_group.append(unclassified)

In [792]:
### FINAL GROUPING
for item in unclassified_group:
    if len(item) != 1:
        temp_grouped = []
        temp_unclassified = []
        temp_grouped.append(item[0])
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:len(item[0])//2] in item[i+1]):
                temp_grouped.append(item[i+1])
            else:
                temp_unclassified.append(item[i+1])
        done.append(temp_grouped)
        #print(temp_unclassified)
        if temp_unclassified != [] and len(temp_unclassified) == 1:
            unique.append(temp_unclassified[0])
    else:
        unique.append(item[0])
#unique 

In [794]:
dict_done = {}
for i in range(len(done)):
    dict_done[i] = done[i]
#dict_done

In [795]:
unique_ = []

for u in unique:
    unique_.append([u])
dict_unique = {i: unique_[j] for i,j in zip([ i for i in range(len(dict_done), len(unique)+len(dict_done))], [j for j in range(len(unique))])}

In [796]:
dict_fb = dict(dict_done)
dict_fb.update(dict_unique)

In [797]:
### MAPPING SAME NAME STORE WITH MAP
store_id = []

for store in list(df_new['stem_store']):
    for key, v in dict_fb.items():
        if store in v:
            store_id.append(key)
            break
        elif (key == list(dict_fb.keys())[-1]):
            store_id.append('9999')

In [799]:
df_new['store_id'] = store_id
df_new = df_new[['post_id','store_id','stores', 'stem_store','create_on', 'ramen_name', 'fb_review']]
df_new.to_csv('store_id_done.csv', index=True)

In [801]:
#load map.csv
df_table_1 = read_csv('Main_Store.csv', encoding='utf8',index_col=0)
map_id = list(df_table_1.index)
map_name = list(df_table_1['main_store'])

In [802]:
fb_match_id = []

for fb_key, fb_v in dict_fb.items():
    fb_v[0] = fb_v[0].lower()
    fb_v[0] = fb_v[0].replace('の','之')
    for m_id, m_name in zip(map_id, map_name):
        m_name = m_name.lower()
        m_name = m_name.replace('の','之')
        if len(fb_v) == 1 and (fb_v[0] == m_name):
            fb_match_id.append(m_id)
            break
        elif (fb_v[0] == m_name):
            fb_match_id.append(m_id)
            break
        elif (len(fb_v) >= 1) and (fb_v[0] in m_name):
            fb_match_id.append(m_id)
            break
        elif (len(fb_v[0]) >= 6) and (fb_v[0][:5] == m_name[:5]):
            fb_match_id.append(m_id)
            #print(m_name[:6])
            break
        elif (len(fb_v[0]) >= 5) and (len(fb_v[0]) <= 7) and (fb_v[0][:4] == m_name[:4]):
            fb_match_id.append(m_id)
            #print(m_name)
            break       
        elif (m_id == 335):
            fb_match_id.append('99999')
            break           

In [810]:
### CREATE STORE-ID
'''
IF the store exsits in Table_1 (Main_Store), store_id matches together;
Otherwise, the store_id is 10000+number, which means this store only exists in FB's Post.
'''
table_foreign_key = []

for store in list(df_new['stem_store']):
    for (fb_k, fb_v), new in zip(dict_fb.items(), fb_match_id):
        if store in fb_v and (new != '99999'):
            table_foreign_key.append(new)
            break
        elif (store in fb_v) and (new == '99999'):
            table_foreign_key.append(fb_k + 10000)
            break
        elif (fb_k == list(dict_fb.keys())[-1]) and (store not in fb_v):
            table_foreign_key.append('99999')
            break

In [805]:
### RE-Create post_id 
post_id = []

for i in range(1,1090):
    if len(str(i)) == 1:
        id_ = 'T3' + '0000' + str(i)
    elif len(str(i)) == 2:
        id_ = 'T3' + '000' + str(i)
    elif len(str(i)) == 3:
        id_ = 'T3' + '00' + str(i)
    elif len(str(i)) == 4:
        id_ = 'T3' + '0' + str(i)
    elif len(str(i)) == 6:
        id_ = 'T3' + str(i)
    post_id.append(id_)
    
df_new['post_id'] = post_id

In [807]:
df_new['store_id'] = table_foreign_key 
df_t3 = df_new[['post_id','store_id','stores', 'create_on', 'ramen_name', 'fb_review']]
df_t3.to_csv('Post.csv', index=False)

In [ ]:
from datetime import datetime

datetime_str = '2020年2月20日'
datetime_str = datetime_str [2:]
print(datetime_str)
datetime_str = datetime_str.replace('年','/').replace('月','/').replace('日','')
print(datetime_str)
# Convert String ( ‘DD/MM/YY HH:MM:SS ‘) to datetime object
datetime_obj = datetime.strptime(datetime_str, '%y/%m/%d')
print(datetime_obj.date())

In [80]:
### Remove emoji in fb_review
import pandas as pd
import emoji
import re
from pandas import read_csv


review_lst = []
df = read_csv('/Users/yuchiacheng/Desktop/FB_Crawling_and_shopname_mapping_with_Map/Tables/Post.csv', encoding='utf8',index_col=0)
review = list(df['fb_review'])


def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ''.join([str for str in text if not any(i in str for i in emoji_list)])
    return clean_text


for r in review:
    r = give_emoji_free_text(r)
    review_lst.append(r)

df['fb_review'] = review_lst
df.to_csv('Post.csv')

In [135]:
import pandas as pd
import emoji
import re
from pandas import read_csv

df = read_csv('/Users/yuchiacheng/Desktop/FB_Crawling_and_shopname_mapping_with_Map/four_city_other_soup_data.csv', encoding='utf8',index_col=0)
store = read_csv('/Users/yuchiacheng/Desktop/FB_Crawling_and_shopname_mapping_with_Map/Tables/Store.csv', encoding='utf8',index_col=0)

taipei = list(df.iloc[0,[9,10,11,12,13,14,15,16]])
kaohsiung = list(df.iloc[1,[9,10]])
tainan = list(df.iloc[2,[9,10,11]])
taichung = list(df.iloc[3,[9,10,11]])

soup = list(store['soup'])
city = list(store['province'])

soup_others= []
for s, c in zip(soup,city):
    s = s.replace('#','')
    if c == '台北市':
        for tag in taipei:
            if tag in s:
                soup_others.append('y')
                break
            elif tag == taipei[-1] and (tag not in s):
                soup_others.append('n')
                break
    elif c == '高雄市':
        for tag in kaohsiung:
            if tag in s:
                soup_others.append('y')
                break
            elif tag == kaohsiung[-1] and (tag not in s):
                soup_others.append('n')
                break
    elif c == '台南市':
        for tag in tainan:
            if tag in s:
                soup_others.append('y')
                break
            elif tag == tainan[-1] and (tag not in s):
                soup_others.append('n') 
                break
    elif  c == '台中市':
        for tag in taichung:
            if tag in s:
                soup_others.append('y')
                break
            elif tag == taichung[-1] and (tag not in s):
                soup_others.append('n') 
                break
    elif (c != '台北市') or (c != '高雄市') or (c != '台南市') or (c != '台中市'):
        soup_others.append('n') 


store['soup_others'] = soup_others
store = store[['store_id','store', 'still_there','address', 'description', 'open_time', 'latitude', 'longtitute',
       'map_review', 'region','province', 'soup', 'soup_others','transport']]
store.to_csv('Store_new.csv')

In [134]:
store

,store_id,store,still_there,address,description,open_time,latitude,longtitute,map_review,region,province,soup,soup_others,transport
detail_store_id,,,,,,,,,,,,,,
T200001,0,Hiro'sらあ麵Kitchen-竹北店,y,302新竹縣竹北市文義街237號,開業於2020年5月，是以豚骨湯頭為主體的拉麵店，目前僅有拉麵及叉燒拉麵二種配置，詳情可察看...,17:30~21:30(Last Order 21:00)週二 公休,24.831140,121.013430,毛文謙 - 叉燒拉麵 https://bit.ly/2VDB0mf 星果汁 - 拉麵...,北,新竹縣,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨,n,文義街
T200002,0,Hiro's らぁ麵Kitchen-中壢店,y,320桃園市中壢區大勇一街35號,開業於2020年4月，是以豚骨湯頭為主體的拉麵店，本店的限定為咖哩拉麵，詳情可察看下方網友食記。,17:30~21:30週一 公休,24.954224,121.240181,星果汁 - 拉麵 http://bit.ly/2EIAXz6,北,桃園市,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨,n,大勇一街
T200003,0,Hiro's らぁ麵 Kitchen-新竹創始店,y,30264新竹縣竹北市文義街237號,開業於2018年10月，是以豚骨湯為主體的拉麵店，2020年10月獲得台北市牛肉麵節紅燒組第...,17:30~21:30營業日不固定，請參考店家的FB粉專,24.831140,121.013430,毛文謙 - 叉燒拉麵 https://bit.ly/2VDB0mf 星果汁 - 拉麵...,北,新竹縣,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #雞清湯,n,世界街、麵屋浩
T200004,1,Le Ming 麵食精釀餐酒館-林口店,y,244新北市林口區南勢街118號,開業於2019年9月，本店的湯體是使用西式手法調理並以干貝雞湯為主體，是目前少見以魚介雞湯系...,17:00 - 21:00週一 公休,25.074141,121.363265,Soron Hsu - 松露干貝雞、番茄雞叉燒沾麵 http://bit.ly/2T9I...,北,新北市,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #雞湯 #雞骨魚介,n,民族路、南勢街交叉口
T200005,1,Le Ming 麵食精釀餐酒館-中山店,y,104台北市中山區中山北路一段135巷1號,開業於2020年5月，店主插旗中山拉麵戰區，本店的湯體是使用西式手法調理並以干貝雞湯為主體，...,17:30~22:00,25.051741,121.522821,何柏賢 - 松露干貝雞拉麵 https://bit.ly/3i3nsbF Yu-Ro...,北,台北市,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #雞白 #干貝雞 #可訂位,n,屯京拉麵
T200006,2,Mr. 拉麵 雲,y,640雲林縣斗六市中堅西路743號,始於2012年10月在台北公館，老闆曾當過記者和東京都議員，在台北念博士班期間開平價拉麵店，...,週一-五 11:00~14:30，17:00~21:00週六-日 11:00~15:00，1...,23.702718,120.531326,無,中,雲林縣,無提供標籤資訊,n,家樂福 斗六店
T200007,3,Ramen 初,y,830高雄市鳳山區五權南路226號,開幕於2016年8月，是高雄在地豚骨系拉麵名店，豚湯濃鮮鹹滑順，不定期推出的限定品項，更是聚...,週一~五 18:00~21:00週六~日 17:30~21:00,22.625422,120.353595,寒沐蝶 - 醬油豚骨拉麵 https://bit.ly/3iXGb9v Huang ...,南,高雄市,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #2019名店,n,仁惠婦幼醫院／捷運鳳山站步行4分
T200008,4,Totto Ramen 鳥人拉麵-西門店,y,108台北市萬華區中華路一段114巷3號,來自紐約的雞湯名店，此間為第三家分店，品項有雞湯、雞湯味噌，與一般少見的，以蔬菜打底的素食拉...,週日~一 : 11:30–22:00週五~六 : 11:30–28:00,25.043942,121.508222,Chih-Hung Weng - 鳥人蔬菜拉麵 https://bit.ly/2MyQa...,北,台北市,#ramentw #台灣拉麵愛好會 #拉麵 #雞白 #咖哩 #2019名店,y,西門町佳佳唱片行／捷運西門站-6號出口步行2分
T200009,4,Totto Ramen 鳥人拉麵-中山店,y,10491台北市中山區中山北路一段105巷7號,來自紐約的雞湯名店，此間為二店，服務的客人大多是以七條通的客人為主，品項有雞湯、雞湯味噌，與...,週日~一 : 11:30~22:00週二~六 : 11:30~28:00,25.050921,121.522884,Chih-Hung Weng - 鳥人蔬菜拉麵 https://bit.ly/2MyQa...,北,台北市,#ramentw #台灣拉麵愛好會 #拉麵 #雞白 #咖哩 #素食 #2019名店,y,新光三越台北南西館／捷運中山站-2號出口


In [136]:
df

,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17
name,,,,,,,,,,,,,,,,,
台北市,豚骨,醬油,雞白,魚介,沾麵,雞清,家系,雞豚,淡麗系,味噌,柑橘,咖哩,煮干,素食,魚白,辣味,拌麵
高雄市,豚骨,醬油,雞白,魚介,沾麵,雞清,家系,雞豚,淡麗系,素食,拌麵,NaN,NaN,NaN,NaN,NaN,NaN
台南市,豚骨,雞白,魚介,沾麵,雞清,雞豚,咖哩,淡麗系,煮干,素食,拌麵,辛口,NaN,NaN,NaN,NaN,NaN
台中市,豚骨,醬油,雞白,魚介,沾麵,雞清,家系,雞豚,味噌,辣味,拌麵,牛肉,NaN,NaN,NaN,NaN,NaN


In [126]:
len(soup_others)

460